## Load query into a dataframe

In [ ]:
from dune_client.client import DuneClient
import pandas as pd
API_KEY = "1EF7SJ7DWZYDkQ0KmbNiutJhtwjQDL5o"
dune = DuneClient(API_KEY)
query_result = dune.get_latest_result(5145525).result.rows #QUERY SQL VISABLE AT https://dune.com/queries/5145525?sidebar=none
print(query_result)
result_table = pd.DataFrame(query_result)
result_table.to_csv('result_table')
result_table.head()

## EDA On Poisoning 

In [ ]:
#most common poisoners
from IPython.display import clear_output
common_poisoners = result_table.value_counts("tx_signature_poisoning")
#volume of transfers from poisoners
attack_volume = result_table.groupby('owner_from_poisoning')['amount_usd_poisoning'].sum()
#get list of transactions to poisoner in next week
query_result = dune.get_latest_result(5149832).result.rows #SQL at https://dune.com/queries/5149832?category=labels&id=labels.owner_addresses&sidebar=none
poisoner_success = pd.DataFrame(query_result)
successful_attacks = poisoner_success.value_counts('owner_from_poisoning')
print(successful_attacks.head())
poisoner_success = poisoner_success.groupby('owner_from_poisoning')['amount_usd_poisoning'].sum()
concat = [poisoner_success, attack_volume]
#show net amnt earned
poisoner_net = pd.concat(concat, join='inner', axis=1)
poisoner_net['net_earning'] = poisoner_net['amount_usd'] - poisoner_net['amount_usd_poisoning']
print(poisoner_net.sort_values('net_earning', ascending=False).head())


